# Tópico 10 - Introdução ao Aprendizado Profundo

O aprendizado profundo consiste em técnicas de aprendizado de máquina que usam Redes Neurais Artificiais. Existem várias arquiteturas possíveis para essas redes. Neste notebook vamos explorar uma Rede Multilayer Perceptron implementada através do Scikitlearn e uma Rede CNN usada para classificação de imagens implementada em Keras/TensorFlow. Ao final, indicamos um link complementar com outros exemplos que podem ser explorados.

-----------------------------

## Exemplo 1 - MLP Sklearn

In [ ]:
# Importando as bibliotecas
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

### (a) Multiplexador Digital:

In [ ]:
# Carregando os dados
dados_multiplexador = pd.read_csv('dados_multiplexador.csv', sep=';')
dados_multiplexador.head(100)

In [ ]:
# Selecionado as variáveis descritivas e a variável alvo
x = dados_multiplexador.drop(columns='Saída')
y = dados_multiplexador['Saída'] 

In [ ]:
# Criando Perceptron
perceptron_multiplexador = Perceptron(tol=1e-3, random_state=0)

In [ ]:
# Treinando o Perceptron
perceptron_multiplexador.fit(x, y)

In [ ]:
# Pesos do percpetron depois do treinamento
perceptron_multiplexador.coef_

In [ ]:
# Valor do bias sendo somado depois do treinamento
perceptron_multiplexador.intercept_

In [ ]:
# Usando o Perceptron para fazer um predição
perceptron_multiplexador.predict([[1,1,1,0,1,1]])

### (b) Porta XOR:

In [ ]:
# Carregando os dados
dados_xor = pd.read_csv('dados_xor.csv', sep=';')
dados_xor.head()

In [ ]:
# Selecionado as variáveis descritivas e a variável alvo
x = dados_xor.drop(columns='XOR')
y = dados_xor['XOR'] 

In [ ]:
# Criando Perceptron e treinando 
perceptron_xor = Perceptron(tol=1e-3, random_state=0)
perceptron_xor.fit(x, y)

In [ ]:
# Usando o Perceptron para fazer um predição
perceptron_xor.predict([[1,0]])

In [ ]:
# Criando Multilayer Perceptron e treinando 
mlp_xor = MLPClassifier(solver='lbfgs', activation='logistic',
                        hidden_layer_sizes=(3, 2), random_state=0)
mlp_xor.fit(x, y)

In [ ]:
# Usando o MLP para fazer um predição
mlp_xor.predict([[1,0]])

--------------------------------
## Exemplo 2 - CNN TensorFlow

### Bibliotecas importadas

In [ ]:
# Bibliotecas gerais
import numpy as np                     # Trabalhar co números
import pandas as pd                    # Trabalhar com tabelas
import matplotlib.pyplot as plt        # Gráficos

# Bibliotecas para construir redes neurais
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator

# Aprendizado de máquina geral - Separação treino/teste, transformação (escalonamento), métricas de desempenho
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, classification_report


from google_drive_downloader import GoogleDriveDownloader as gdd

## Classificação de Imagens  com Redes Convolucionais

Vamos usar o dataset CIFAR 10. Ver https://en.wikipedia.org/wiki/CIFAR-10. 

O CIFAR 10 é um dataset de classificação de imagens com as seguintes classes:

*   Aviões
*   Carros
*   Pássaros
*   Gatos
*   Veados
*   Cachorros
*   Sapos
*   Cavalos
*   Návios
*   Caminhões

Queremos criar um sistema de classificação automático para imagens, onde dada uma imagem de entrada, classificamos ela entre uma das 10 classes anteriores.

<center>
<img src='https://drive.google.com/uc?id=1yIihwCkW511n8A4o-JMfLjD4KKJw7m9a'width=700>

### Carregando as imagens

Perceba que já estamos separando elas em treino e teste. 

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [ ]:
for i in range(9):  
    plt.subplot(330 + 1 + i) # define subplot
    plt.imshow(train_images[i]) # plot raw pixel data
# show the figure
plt.show()

In [ ]:
len(train_images), len(train_labels), len(test_images), len(test_labels)

Temos um total de 50000 imagens diferentes com dimensão de 32x32x3 pixels

In [ ]:
train_images.shape

### Pré-processamento dos rótulos e imagens

Vamos transformar os rótulos que estão em arrays numéricos em uma matriz de categorias.

Ver https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical.

In [ ]:
train_labels

In [ ]:
train_labels = tf.keras.utils.to_categorical(train_labels)
train_labels.shape

In [ ]:
test_labels = tf.keras.utils.to_categorical(test_labels)
test_labels.shape

Em uma imagem RGB, a intesidade de cada cor tem um valor inteiro entre 0 e 255 ($2^8$ - ou seja, 8 bits de codificação por valor). Vamos normalizar esse valor para ficar entre 0 e 1.

Lembre-se, cada imagem é uma matriz de números (no caso RGB, três matrizes). Cada posição da matriz é chamada de pixel, e possui um valor de intensidade. Ao dividir a matriz por um mesmo número, estamos divindo o valor de cada pixel por esse número.

In [ ]:
# Normalizando os valores dos pixel para serem entre 0 e 1
train_images, test_images = train_images / 255.0, test_images / 255.0

### Criando a Arquitetura da Rede Neural Artificial

#### Como funciona uma Rede Neural Convolucional

Para processamento de imagens podemos usar **redes neurais convolucionais**. Esse tipo de rede tem seus parâmetros em uma matriz (*kernel*) que processa toda a imagem passando por todos os seus pixels. Essa matriz são os parâmetros que serão aprendidos pela rede. Esse procedimento de multiplicação entre parte da imagem e a matriz de kernel é semelhante a uma operação de convolução de sinais. Daí o nome do método.

<center>
<img src='https://drive.google.com/uc?id=15kidvb-4BozzWN9_AfIbxKJftezkKK4U'width=1000>


No exemplo da imagem acima, podemos perceber que a convolução é uma operação que pode reduzir a dimensão da entrada, uma vez que tinhamos uma imagem de 7x7 pixels na entrada e como saída final ficamos com 5x5 pixels.

Neste exemplo temos uma convolução ocorrendo numa imagem com um único canal. Entretanto, uma das formas de representações é o espaço de cores RGB. A imagem abaixo ilustra o exemplo da clássica imagem da lenna nesse espaço de cores:

<center>
<img src='https://drive.google.com/uc?id=1S-oIn5WmGyr6Xg5Cc4hs4PvCrncXoZw2'width=700>



A união dos três canais forma a imagem colorida (256x256x3). Dessa forma a convolução irá operar nos três canais, tendo um *kernel* para cada canal:

<center>
<img src='https://drive.google.com/uc?id=1KALsA9iuIDkaURYEmgu1XD69hz04siyG'width=700>

O exemplo acima é aplicado quande deseja-se transformar os três canais de convolução em uma única saída.

No nosso modelo de Rede Neural, vamos conectar sequencialmente diferentes camadas com diferentes propósitos. Após a primeira camada de entrada, vamos passar por duas camadas de Convolução.

Perceba que as funções de ativação das camadas de convolução é uma função tipo ReLu. Na primeira camada densa, também, e na última camada da rede, é uma função Softmax já que queremos fazer classificação.

In [ ]:
modelo = tf.keras.Sequential()

modelo.add(Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3))) # Entra com imagem 32,32,3
modelo.add(Conv2D(32, (3, 3), activation='relu')) 
modelo.add(MaxPooling2D((2, 2),))
modelo.add(Conv2D(32, (3, 3), activation='relu'))
modelo.add(Flatten())
modelo.add(Dense(16, activation='relu'))
modelo.add(Dense(10, activation='softmax')) 



modelo.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]) # Métricas para treinar

Na sequência das camadas de Convolução, temos uma camada de **Max Pooling**. Essa camada é utilizada para fazer com que a rede seja invariante a pequenas alterações na posição da imagem nos pixels de entrada.

Ver https://keras.io/api/layers/pooling_layers/max_pooling2d/

<center>
<img src='https://drive.google.com/uc?id=1G96qhP-Z6piTvWrPopfGxzfkvSthNPOO'width=700>

Temos também a camada de **Flatten** que apenas irá transformar a saída da convolução que é bidimensional em uma dimensão. Isso é feito para que posteriormente seja possível implementar uma **rede neural densa** (como a rede Multilayer Perceptron), já que esse espera como entrada um vetor:

<center>
<img src='https://drive.google.com/uc?id=1jG7b_oFVFdFSXwcOEuMabzKb7SpvXZrb'width=500>

In [ ]:
modelo.summary()

In [ ]:
plot_model(modelo, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

### Realizando o treinamento

In [ ]:
# Vamos fazer o treinamento utilizando 35 epocas!
history = modelo.fit(train_images,train_labels,epochs=30, verbose=1, batch_size=120, validation_split=0.2)

In [ ]:
#modelo.save('CNN_Classificador.h5')

In [ ]:
# Plotando o MSE para cada época de treinamento
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Função de Custo vs Época')
plt.ylabel('Função de Custo')
plt.xlabel('Épocas')
plt.legend(['treino', 'validação'], loc='upper right')
plt.show()

Podemos perceber que nesse primeiro treinamento houve um overfitting, uma vez que o confjunto de validação após 10 épocas de treinamento começou a aumentar o valor da sua função custo

## Criando um novo modelo

Agora vamos criar um segundo novo modelo para tentar conseguir um resultado diferente no treinamento:

In [ ]:
model= tf.keras.Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(32,32,3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(64, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(Dropout(0.2)) 
model.add(Conv2D(128, (3, 3), activation='relu', padding='same')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Flatten()) 
model.add(Dropout(0.2)) 
model.add(Dense(1024, activation='relu')) 
model.add(Dropout(0.2)) 
model.add(Dense(512, activation='relu')) 
model.add(Dropout(0.2)) 
model.add(Dense(10, activation='softmax'))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

O **padding** é uma forma de realizar a convolução sem alterar a dimensão da saída da operação de convolução.

Ver https://keras.io/api/layers/convolution_layers/convolution2d/

<center>
<img src='https://drive.google.com/uc?id=1pxfS3V9DquLJNIvx3LupZz12KBm5NtIO'width=800>

O **Dropout** é uma técnica que faz com que alguns neurônios seja desativados aleatoriamente durante o treinamento. O objetivo disso é que evitar o Overfitting. Seria o similar a treinarmos diferentes redes e obtermos a média delas como saída. No caso o dropout irá eliminar 20% dos neurônios durante o treinamento.

Realizando o novo treinamento;

In [ ]:
# Vamos fazer o treinamento utilizando 10 epocas!
history_2 = model.fit(train_images,train_labels,epochs=10, verbose=1, batch_size=120, validation_split=0.2)

In [ ]:
model.save('CNN_Classificador_2.h5')

In [ ]:
# Plotando o MSE para cada época de treinamento
plt.plot(history_2.history['loss'])
plt.plot(history_2.history['val_loss'])
plt.title('Função de Custo vs Época')
plt.ylabel('Função de Custo')
plt.xlabel('Épocas')
plt.legend(['treino', 'validação'], loc='upper right')
plt.show()

### Avaliando o desempenho do classificador

Agora, usando a porção de dados de teste, iremos realizar as previsões de todos os valores de saída.

Vamos utilizar o modelo ser Overfitting.

In [ ]:
novo_modelo = load_model('CNN_Classificador_2.h5')

In [ ]:
y_pred = novo_modelo.predict(test_images).argmax(axis=1)
y_pred

In [ ]:
cm_rn = confusion_matrix(test_labels, y_pred, labels = [0,1,2,3,4,5,6,7,8,9])
cm_rn

In [ ]:
figure = plt.figure(figsize=(30, 20))
disp = ConfusionMatrixDisplay(confusion_matrix = cm_rn, display_labels=[0,1,2,3,4,5,6,7,8,9])
disp.plot(values_format='d') 

In [ ]:
# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(test_labels, y_pred))

------------------------
# Outros exemplos:

No link https://github.com/hfarruda/deeplearningtutorial há uma gama de notebooks para diferentes arquiteturas de redes neurais. Vamos explorar alguns deles.